<h1><center>ЛОГИРУЕМ МОДЕЛЬ</h1></center>

Пора продолжить работу над проблемой оттока клиентов! После первой итерации вы получили базовое решение — модель логистической регрессии. Вы также провели свой первый эксперимент и залогировали основные артефакты и сам набор данных.
<br><br>Прежде чем перейти к логированию модели в MLflow, важно убедиться, что вам доступна сама модель, а MLflow настроен в соответствии с предыдущими уроками. 
<br><br>
1.(**команда sh run_mlflow_server.sh делает все шаги первого этапа**) Поднимите MLflow, используя предложенный ранее вариант конфигурации:

- локальный Tracking Server на виртуальной машине,
- удалённые хранилище экспериментов и хранилище артефактов.

Не забудьте настроить соответствующие переменные окружения для подключения к базе данных Postgres и объектному хранилищу S3.

2.(**код в этом блокноте**) Определите необходимые глобальные переменные. Одной из них будет EXPERIMENT_NAME, которую вы уже использовали. Вторую переменную, RUN_NAME, вы можете назвать самостоятельно, чтобы ясно отразить суть запуска.


In [ ]:
import pandas as pd
import numpy as np
import os
import mlflow
import joblib
from mlflow.tracking import MlflowClient
import random
from sklearn.model_selection import train_test_split

# определяем основные credentials, которые нужны для подключения к MLflow
# важно, что credentials мы передаём для себя как пользователей Tracking Service
# у вас должен быть доступ к бакету, в который вы будете складывать артефакты
os.environ["MLFLOW_S3_ENDPOINT_URL"] = "https://storage.yandexcloud.net" #endpoint бакета от YandexCloud
os.environ["AWS_ACCESS_KEY_ID"] = 'YCAJE3Nlz8iDILW5VTYM1ihQB' # os.getenv("AWS_ACCESS_KEY_ID") # получаем id ключа бакета, к которому подключён MLFlow, из .env
os.environ["AWS_SECRET_ACCESS_KEY"] = 'YCPjvS7uwhvJpUj3bKm8X-IX4QAwBIVsvX61IL44' # os.getenv("AWS_SECRET_ACCESS_KEY") #  получаем ключ бакета, к которому подключён MLFlow, из .env

# определяем глобальные переменные
# поднимаем MLflow локально
TRACKING_SERVER_HOST = "127.0.0.1"
TRACKING_SERVER_PORT = 5000

YOUR_NAME = "unique_experiment-5" # введите своё имя для создания уникального эксперимента
assert YOUR_NAME, "alexnikzotov"

# название тестового эксперимента и запуска (run) внутри него
EXPERIMENT_NAME = f"test_connection_experiment_{YOUR_NAME}"
RUN_NAME = "test_connection_run"

# тестовые данные
METRIC_NAME = "test_metric"
METRIC_VALUE = 0

# устанавливаем host, который будет отслеживать наши эксперименты
mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")

# создаём тестовый эксперимент и записываем в него тестовую информацию
experiment_id = mlflow.create_experiment(EXPERIMENT_NAME)
with mlflow.start_run(run_name=RUN_NAME, experiment_id=experiment_id) as run:
    run_id = run.info.run_id
    
    mlflow.log_metric(METRIC_NAME, METRIC_VALUE) 

3.Для логирования используйте либо уже имеющуюся модель, полученную в первом спринте, либо обучите новую модель по своему усмотрению - **обучим новую модель**. Загрузим и подготовим данные:


In [18]:
df = pd.read_csv('users_churn.csv')
print('Колво строк до обработки:',df.shape[0])

# Закодируем целевую переменную
df['target'] = (df['end_date'].notna()).astype(int)

# Выделим признаки в три отдельные таблицы для дальнейшей работы:
features = df.drop(['customer_id','target'],axis=1)
num_features = features.select_dtypes(include=['float', 'int'])
date_features = features.select_dtypes(include='datetime64[ns]')
cat_features = features.select_dtypes(include='object')

# Посчитаем колво уникальных значений для катег. переменных и создадим создадим два датасета:
unique_values_per_col = cat_features.nunique().value_counts()
binary_cat_features = cat_features[ [i for i in cat_features.columns if cat_features[i].nunique()==2] ]
other_cat_features = cat_features[ [i for i in cat_features.columns if cat_features[i].nunique()!=2] ]

# Бинарные подразделяем на два - "да/нет" и другие бинарные:
yes_no_features = binary_cat_features[ [i for i in binary_cat_features.columns if\
binary_cat_features[i].isin(['Yes','yes','No','no',None,np.nan]).all()==True] ]
other_binary_features = binary_cat_features[ [i for i in binary_cat_features.columns if\
binary_cat_features[i].isin(['Yes','yes','No','no',None,np.nan]).all()!=True] ]

# Дубликаты
is_duplicated_id = df.duplicated(subset=['customer_id'], keep=False)

# Пропуски
cols_with_nans = df.isnull().sum()
cols_with_nans = cols_with_nans[cols_with_nans > 0].index.drop('end_date')
for col in cols_with_nans:
    if df[col].dtype in [float, int]:
        fill_value = df[col].mean()
    elif df[col].dtype == 'object':
        fill_value = df[col].mode().iloc[0]
    df[col] = df[col].fillna(fill_value)

# Выбросы
num_cols = df.select_dtypes(['float']).columns
threshold = 1.5
potential_outliers = pd.DataFrame()
for col in num_cols:
	Q1 = df[col].quantile(0.25)
	Q3 = df[col].quantile(0.75)
	IQR = Q3 - Q1
	margin = threshold * IQR
	lower = Q1 - margin
	upper = Q3 + margin
	potential_outliers[col] = ~df[col].between(lower, upper)
outliers = potential_outliers.any(axis=1)

df.drop(columns=['id', 'customer_id', 'begin_date', 'end_date'], inplace=True)
df[-3:]


Колво строк до обработки: 7019


,type,paperless_billing,payment_method,monthly_charges,total_charges,internet_service,online_security,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,gender,senior_citizen,partner,dependents,multiple_lines,target
7016,Month-to-month,Yes,Electronic check,29.60,346.45,DSL,Yes,No,No,No,No,No,Female,0,Yes,Yes,No,0
7017,Month-to-month,Yes,Mailed check,74.40,306.60,Fiber optic,No,No,No,No,No,No,Male,1,Yes,No,Yes,1
7018,Two year,Yes,Bank transfer (automatic),105.65,6844.50,Fiber optic,Yes,No,Yes,Yes,Yes,Yes,Male,0,No,No,No,0


Обучим модель:

In [ ]:
# Базовый код, который нужно будет использовать после обучения
# model = joblib.load('/home/mle-user/mle_projects/mle-mlflow/models/model.pkl')
# experiment_id = '4'
# with mlflow.start_run(run_name='log_model', experiment_id=experiment_id) as run:
#     run_id = run.info.run_id
  
#     model_info = mlflow.catboost.log_model(cb_model=model, artifact_path="models") 

4.Установите библиотеку scikit-learn в ваше окружение: pip install scikit-learn==1.3.1.




Также для успешного выполнения заданий подготовьте следующие пререквизиты:
- Объект модели. Это та самая обученная модель, которую вы планируете логировать с помощью MLflow. Сохраните её в переменную model.
- Предсказанные моделью значения. Полученные прогнозы модели, включая вероятности положительного исхода и бинарные значения, которые вы определили. - Сохраните эти значения в переменные: prediction — бинарные значения 0 или 1, proba — вероятности.
- Истинные метки и данные для предсказания. Первое — это истинные значения целевой переменной для вашего тестового набора данных. Сохраните их в переменную y_test. Второе — датасет с признаками для модели, который уже сохранён в переменную X_test.

In [ ]:
# Загрузим данные:
df = pd.read_csv('users_churn.csv') # проблема здесь - нужны сразу предлобработанные данные

# Поделим данные:
X = df.drop('target',axis=1)
y = df['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Получение бинарных значений
prediction = model.predict(X_test)

# Получение вероятностей положительного исхода
proba = model.predict_proba(X_test)